# NHANES data

> ### [Variable search](http://wwwn.cdc.gov/Nchs/Nhanes/Search/default.aspx)
> ### [Demographic data](http://wwwn.cdc.gov/Nchs/Nhanes/Search/DataPage.aspx?Component=Demographics) - [variable list](http://wwwn.cdc.gov/nchs/nhanes/search/variablelist.aspx?Component=Demographics)
> ### [Dietary](http://wwwn.cdc.gov/Nchs/Nhanes/Search/DataPage.aspx?Component=Dietary) - [variable list](http://wwwn.cdc.gov/nchs/nhanes/search/variablelist.aspx?Component=Dietary)
> ### [Examination](http://wwwn.cdc.gov/Nchs/Nhanes/Search/DataPage.aspx?Component=Examination) - [variable list](http://wwwn.cdc.gov/nchs/nhanes/search/variablelist.aspx?Component=Examination)
> ### [Laboratory](http://wwwn.cdc.gov/Nchs/Nhanes/Search/DataPage.aspx?Component=Laboratory) - [variable list](http://wwwn.cdc.gov/nchs/nhanes/search/variablelist.aspx?Component=Laboratory)
> ### [Questionnaire](http://wwwn.cdc.gov/Nchs/Nhanes/Search/DataPage.aspx?Component=Questionnaire) - [variable list](http://wwwn.cdc.gov/nchs/nhanes/search/variablelist.aspx?Component=Questionnaire)

# Methods

### Data source
> In this study, we used a 1999-2004 data set from the National Health and Nutrition Examination Survey (NHANES) to generate the SVM algorithm. NHANES is an ongoing, cross-sectional, probability sample survey of the U.S. population. It collects demographic, health history, and behavioral information from participants in home interviews. Participants are also invited for detailed physical, physiological, and laboratory examinations that are performed by trained personnel in specially equipped mobile centers [15].

> We limited our study to non-pregnant participants aged 20 or older. Participants were considered to have diagnosed diabetes if they answered “yes” to the question “Have you ever been told by a doctor or health professionals that you have diabetes?” Participants who answered “no” to this question but who had a measured fasting plasma glucose ≥ 126 mg/dl were considered to have undiagnosed diabetes; those with a fasting plasma glucose 100-125 mg/dl were considered to have pre-diabetes. Participants with fasting glucose <100 mg/dl were considered to not have diabetes

> We devised two different classification schemes (Table 1). In Classification Scheme I, the group of persons with diabetes (diagnosed or undiagnosed) was distinguished from those without diabetes, including persons with pre-diabetes. In Classification Scheme II, the group of persons with either undiagnosed diabetes or pre-diabetes was distinguished from those without diabetes. The models were developed using a sample of 80% of the individuals in each group and validated in the remaining 20%.

### Variable selection
> We selected 14 simple variables commonly associated with the risk for diabetes: family history, age, gender, race and ethnicity, weight, height, waist circumference, BMI, hypertension, physical activity, smoking, alcohol use, education, and household income. Variable selection was performed according to an automatic approach developed by Chen et al. [16]. The significance of the automatically selected set of variables was further manually evaluated by fine tuning parameters. The variables included in the final selection were those with the best discriminative performance.

### Table 1

<img src="./img/table1.png">

In [1]:
# from utils import fetch_NHANES
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import matplotlib.pylab as plt
from nhanes.merge_data import merge_data

plt.style.use('ggplot')
%matplotlib inline

In [2]:
merge_data('./data/diabetes_data.csv')

In [3]:
df = pd.read_csv('./data/diabetes_data.csv')

In [4]:
df.head()

,ALQ110,ALQ120Q,BMXBMI,BMXHT,BMXWAIST,BPQ020,DMDEDUC2,INDHHINC,MCQ250A,PAQ180,RIAGENDR,RIDAGEYR,RIDRETH1,SEQN,SMD130,SMQ050Q,WTINT2YR,status
0,NaN,1.000000e+00,25.57,157.7,NaN,1,1,3,1,1,1,70,1,13,NaN,20,1647.446107,1
1,NaN,5.397605e-79,36.94,174.9,130.6,1,3,3,1,1,1,62,3,29,1.600000e+01,3,42515.710172,1
2,1,1.000000e+00,26.17,171.1,94.9,2,2,6,1,4,1,59,3,130,5.397605e-79,6,70420.818338,1
3,NaN,3.000000e+00,29.51,168.3,110.6,2,3,3,9,1,1,82,4,266,NaN,NaN,6916.692889,1
4,NaN,5.397605e-79,22.36,169.3,91.3,2,4,6,1,3,1,56,3,272,5.397605e-79,17,95894.532170,1


In [5]:
df.apply(lambda x: x.isnull().sum(), axis=0)

ALQ110      4143
ALQ120Q      909
BMXBMI       122
BMXHT         93
BMXWAIST     155
BPQ020        56
DMDEDUC2       0
INDHHINC     517
MCQ250A        0
PAQ180         0
RIAGENDR       0
RIDAGEYR       0
RIDRETH1       0
SEQN           0
SMD130      4840
SMQ050Q     3961
WTINT2YR       0
status         0
dtype: int64

### Impute values

In [ ]:
# df.INDHHINC.value_counts().index[0]

# df.BPQ020.plot(kind='hist')

# df.loc[df.BMXBMI.isnull(), ['BMXBMI']] = df.BMXBMI.mean()
# df.loc[df.BMXHT.isnull(), ['BMXHT']] = df.BMXHT.mean()
# df.loc[df.BMXWAIST.isnull(), ['BMXWAIST']] = df.BMXWAIST.mean()

# df.BMXBMI.mean()

In [ ]:
df_shuff = df.reindex(np.random.permutation(df.index))

In [ ]:
df_shuff.status.value_counts()

In [ ]:
test_y = df_shuff[:500].status
test_X = df_shuff[:500].drop(['status', 'ALQ110'], axis=1).fillna(0)

train_y = df_shuff[500:].status
train_X = df_shuff[500:].drop(['status', 'ALQ110'], axis=1).fillna(0)

In [ ]:
clf = LogisticRegression().fit(train_X, train_y)

In [ ]:
pred_proba = clf.predict_proba(test_X)
pred = clf.predict(test_X)

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(
    np.where(test_y == 'diag', 1, 0), 
    pred_proba[:,0]
)

# Performance

$$\mathit{Precision = PPV} = \frac {\mathit{TP}} {\mathit{TP} + \mathit{FP}}$$

$$\mathit{Recall = Sensitivity = TPR} = \frac {\mathit{TP}} {P} = \frac {\mathit{TP}} {\mathit{TP}+\mathit{FN}}$$

In [ ]:
print(classification_report(test_y, pred))

In [ ]:
plt.figure(figsize=(8,8))
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.01])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.show()